<a href="https://colab.research.google.com/github/Yash-Savaliya-242/Tensorflow-and-PyTorch-lab03-/blob/main/Lab03_TensorFlow_vs_PyTorch_Yash_Savaliya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 03: TensorFlow vs. PyTorch
- Train a model on MNIST in both TensorFlow and PyTorch, convert to TFLite and ONNX.  
- Use tf.GradientTape for Tensorflow custom training loop.



## TensorFlow Implementation

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalize pixel values
x_test = x_test / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),        # MNIST image shape
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),  # Hidden layer with 128 neurons
    tf.keras.layers.Dense(10, activation='softmax')  # Output layer for 10 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',       # Multi-class classification loss
              metrics=['accuracy'])

# Train model
start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end - start:.2f} seconds")
model.evaluate(x_test, y_test)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.8845 - loss: 0.4287
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9642 - loss: 0.1225
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9770 - loss: 0.0782
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9829 - loss: 0.0569
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9877 - loss: 0.0418
TF Training time: 49.20 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9732 - loss: 0.0925


[0.08076023310422897, 0.9757999777793884]

## Convert TensorFlow model to TFLite

In [2]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpv_6rukwi'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  136739725438096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136739725442896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136739725441744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136739725438672: TensorSpec(shape=(), dtype=tf.resource, name=None)


## PyTorch Implementation

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Data loaders
transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_loader = DataLoader(datasets.MNIST(root='./data', train=True, transform=transform, download=True), batch_size=32)
test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transform, download=True), batch_size=1000)

# Model definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

# Training loop
start = time.time()
for epoch in range(5):
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

# Evaluation
model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()
print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")

100%|██████████| 9.91M/9.91M [00:00<00:00, 58.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.70MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.8MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.43MB/s]


PyTorch Training time: 64.67 seconds
Test accuracy: 0.9723


## Convert PyTorch model to ONNX

In [4]:
# Install ONNX
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 75.0 MB/s eta 0:00:00


In [6]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Fill in normalization factor
x_test = x_test / 255.0   # Fill in normalization factor
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32         # Fill same batch size as in first TF example
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3148, Accuracy: 0.1875
Step 100, Loss: 0.3813, Accuracy: 0.7710
Step 200, Loss: 0.3963, Accuracy: 0.8291
Step 300, Loss: 0.2935, Accuracy: 0.8543
Step 400, Loss: 0.2359, Accuracy: 0.8699
Step 500, Loss: 0.6055, Accuracy: 0.8803
Step 600, Loss: 0.1140, Accuracy: 0.8887
Step 700, Loss: 0.2442, Accuracy: 0.8947
Step 800, Loss: 0.1722, Accuracy: 0.8999
Step 900, Loss: 0.1052, Accuracy: 0.9045
Step 1000, Loss: 0.0741, Accuracy: 0.9082
Step 1100, Loss: 0.1825, Accuracy: 0.9108
Step 1200, Loss: 0.1734, Accuracy: 0.9138
Step 1300, Loss: 0.5663, Accuracy: 0.9163
Step 1400, Loss: 0.1533, Accuracy: 0.9184
Step 1500, Loss: 0.1344, Accuracy: 0.9207
Step 1600, Loss: 0.1593, Accuracy: 0.9230
Step 1700, Loss: 0.4607, Accuracy: 0.9245
Step 1800, Loss: 0.1223, Accuracy: 0.9268
Training Accuracy for epoch 1: 0.9281

Epoch 2/5
Step 0, Loss: 0.1387, Accuracy: 0.9375
Step 100, Loss: 0.1925, Accuracy: 0.9570
Step 200, Loss: 0.0468, Accuracy: 0.9613
Step 300, Loss: 0.0631, Accuracy:

## Performance Otimization with Graph Execution using @tf.function

In [8]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Fill in normalization factor
x_test = x_test / 255.0   # Fill in normalization factor
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compile the function into a graph
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.4893, Accuracy: 0.1250
Step 100, Loss: 0.4617, Accuracy: 0.7559
Step 200, Loss: 0.5233, Accuracy: 0.8186
Step 300, Loss: 0.1939, Accuracy: 0.8491
Step 400, Loss: 0.2252, Accuracy: 0.8660
Step 500, Loss: 0.1000, Accuracy: 0.8765
Step 600, Loss: 0.1967, Accuracy: 0.8852
Step 700, Loss: 0.3650, Accuracy: 0.8920
Step 800, Loss: 0.1205, Accuracy: 0.8971
Step 900, Loss: 0.1424, Accuracy: 0.9015
Step 1000, Loss: 0.2553, Accuracy: 0.9054
Step 1100, Loss: 0.0705, Accuracy: 0.9087
Step 1200, Loss: 0.1668, Accuracy: 0.9115
Step 1300, Loss: 0.5505, Accuracy: 0.9138
Step 1400, Loss: 0.1903, Accuracy: 0.9161
Step 1500, Loss: 0.2177, Accuracy: 0.9185
Step 1600, Loss: 0.1007, Accuracy: 0.9209
Step 1700, Loss: 0.0800, Accuracy: 0.9235
Step 1800, Loss: 0.1221, Accuracy: 0.9254
Training Accuracy for epoch 1: 0.9267

Epoch 2/5
Step 0, Loss: 0.0487, Accuracy: 1.0000
Step 100, Loss: 0.1055, Accuracy: 0.9638
Step 200, Loss: 0.2141, Accuracy: 0.9616
Step 300, Loss: 0.1869, Accuracy: